# Run REopt API located on localhost or NREL server

## Initialization

In [2]:
%matplotlib widget
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import copy
import os
from src.post_and_poll import get_api_results

In [3]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

## Load a previously saved API response .json file instead of running REopt

In [4]:
response_json = 'results'
with open('outputs/' + response_json + '.json', 'rb') as handle:
    api_response = json.load(handle)

## Scenario Inputs (POST), if wanting to do a new API call

In [2]:
post_1 = {
  "Scenario": {
    "timeout_seconds": 400,
    "optimality_tolerance_techs": 0.001,
    "time_steps_per_hour": 1,
    "Site": {
      "latitude": 37.78,
      "longitude": -122.45,
      "LoadProfile": {
        "doe_reference_name": "Hospital",
      },  
      "ElectricTariff": {
        "net_metering_limit_kw": 0.0,
        "interconnection_limit_kw": 100000000.0,
        "urdb_label": "5e1676e95457a3f87673e3b0"
      },
      "PV": {
        "max_kw": 1000.0
      },
      "Storage": {
        "max_kw": 500.0,
        "max_kwh": 1000.0
      }
    }
  }
}

## Save the POST to a .json file for sharing or future loading in

In [13]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
with open("inputs/" + 'post_1.json', 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the inputs/POST

In [4]:
# Load a json into a python dictionary
load_post = "post_1"
with open("posts/" + load_post + ".json", 'r') as fp:
    post_1 = json.load(fp)

### Enter the root_url based on the "location" of the server that's hosting the API
### and your API key from developer.nrel.gov

In [5]:
# Enter the branch name here, only used for development/TestRanch and staging servers
branch_name = "ghp"

# Your API Key, only needed for production server
API_KEY = "uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo"

# Local host, DO NOT use the first one if hosting API on Docker
#root_url = "http://localhost:8000/v1"  # Cannot access host localhost from "within Docker container" as this is configured
#root_url = "http://host.docker.internal:8000/v1"  # Must use this or below (gateway.docker.internal) for accessing host localhost
root_url = "http://gateway.docker.internal:8000/v1"  # This one **might** work for both linux and Windows - how about WSL2?

# Development/TestRanch server
#root_url = "https://" + branch_name + "-reopt-dev-api.its.nrel.gov/v1"

# Staging server
#root_url = "https://" + branch_name + "-reopt-stage-api.its.nrel.gov/v1"

# Production server
root_url = "https://developer.nrel.gov/api/reopt/v1"

## POST and poll (periodic GET request) API to GET a new result, if not loading in a previous response
## ...This may take a while!

`get_api_results` POST's your JSON to the API `job` endpoint, which provides a `run_uuid` if the input is valid, and then polls the `results` endpoint using the `run_uuid` until the results come back with a status other than `Optimizing...`.

`get_api_results` also saves the results (full API response, including inputs) to the `results_file`.

A log file is also created in the current working directory.

In [6]:
api_response = get_api_results(post=post_1, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file='responses/results_file.json', 
                               run_id=None)

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo.
main         INFO     Polling https://developer.nrel.gov/api/reopt/v1/job/0a3a7479-3471-4542-ade6-fd3357f9ec33/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo for results with interval of 5s...
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packag

### If you get disconnected from the polling function but you think it ran, 
### copy the run_uuid from the log above to manually GET the results:

In [10]:
run_uuid = "10816148-36c0-4694-a96b-906416c0cc22"
results_url = root_url + '/v1/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

## Get summary of results

In [8]:
print("NPV ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["npv_us_dollars"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotTES", "ColdTES", "AbsorptionChiller", "GHP", "NewBoiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1["Scenario"]["Site"].keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        # PV and Storage are considered if the POST does not explicitly make max_[size] == 0
        if tech == "PV" and post_1["Scenario"]["Site"]["PV"]["max_kw"] == 0:
            pass
        elif tech == "Storage" and (post_1["Scenario"]["Site"]["Storage"]["max_kw"] == 0 or post_1["Scenario"]["Site"]["Storage"]["max_kwh"] == 0):
            pass
        else:
            for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])
    elif tech in ["PV", "Storage"]:
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])                

NPV ($) =  1147830.0
Capital Cost, Net ($) =  1291312.94
PV size_kw  =  1000.0
Storage size_kw  =  184.81066669522215
Storage size_kwh  =  690.3810838558549


### Here are some results keys examples:

In [7]:
api_response.keys()

dict_keys(['outputs', 'inputs', 'messages'])

In [9]:
api_response["outputs"]["Scenario"]["status"]

'optimal'

In [11]:
for k in api_response["outputs"]["Scenario"]["Site"].keys():
    print(k)

year_one_emissions_lb_C02
year_one_emissions_bau_lb_C02
outdoor_air_temp_degF
renewable_electricity_energy_pct
Financial
LoadProfile
LoadProfileBoilerFuel
LoadProfileChillerThermal
ElectricTariff
FuelTariff
Storage
Generator
Wind
CHP
Boiler
ElectricChiller
AbsorptionChiller
HotTES
ColdTES
NewBoiler
SteamTurbine
GHP
PV


## Do some custom response/results data processing

## Save API response and scenario data into a .pickle file for later use

In [27]:
response_save = api_response
file_name_to_save = "response_1"
with open("saved_responses/" + file_name_to_save + ".json", 'w') as fp:
    json.dump(response_save, fp)